# 多进程和多线程
- 程序：一堆代码以文本形式存入一个文档
- 进程：程序运行的一个状态，每一个进程提供执行程序所需要的所有资源（进程本质是资源的集合）
    
    - 包含地址空间，可执行的代码，操作系统的接口，安全的上下文（记录保存启动该进程的用户和权限），唯一的进程ID，内存空间，数据栈等，还有至少有一个线程。
    - 每一个进程启动时都会最先产生一个线程，即主线程，然后主线程再创建其他的子线程。
    - 每个进程有自己完全独立的运行环境，多进程数据不共享，除非采用一定的方法
- 线程：操作系统能够进行运算调度的最小单位，被包含在进程中，是进程的实际运作单位
    - 一个线程指的是进程中一个单一顺序的控制流，一个进程中可有多个并发线程
    - 一个线程就是一个执行上下文
- 进程与线程的区别：
    - 同一个进程中的线程共享同一内存空间，进程之间相互独立
    - 同一个进程中的所有线程的数据是共享的（进程通讯），进程之间的数据是独立的
    - 对主线程的修改可能会影响其他线程的行为，但是父进程的修改（除非删除）不会影响子进程
    - 线程是一个上下文的执行指令，进程则是运算相关的资源集合
    - 同一个进程的线程之间可以直接通信，但是进程之间的交流需要借助中间代理来实现
    - 创建新的线程很容易，但是创建新的进程需要对父进程做一次复制
    - 一个线程可以操作同一进程的其他线程，但是进程只能操作其子进程
    - 线程启动速度快，进程启动速度慢（但是两者的运行速度没有可比性）

# 多线程
- Python多线程的工作过程
    - Python在使用多线程的时候，调用的是C语言的原生线程
        - 1.拿到共有数据
        - 2.申请GIL
        - 3.Python解释器调用os原生线程
        - 4.os操作CPU执行运算
        - 5.当该线程执行时间到后，无论运算是否执行完，GIL都内要求释放
        - 6.GIL被另一个线程拿到，重复上述操作
        - 7.等到其他线程执行完毕，又会切换到之前的线程（根据其保存的执行上下文继续执行。Python3中由程序计数器改成程序计时器，时间一到，GIL就释放
        
# Python实现多线程的方法
- python2中，采用thread方法，Python3中_thread，threading在两者中通用
### 创建线程常用的两种方法
- threading
    - 直接利用threading.Thread生成Thread实例
        - t = threading.Thread(target=func_name, args=元组），元组内容是前面函数所要输入的参数。注意，只有一个参数时末尾要加逗号如：args = (age,)
- 继承threading.Thread,其本质是重写Thread类中的run函数以及__init__方法
### threading.Thread()提供的线程对象方法和属性
- start()：创建线程后通过start启动线程，等待CPU调度，为run函数执行做准备；
- ident：整数类型的线程标识符，线程开始执行前（调用start之前）为None；
- isAlive()、is_alive()：测试线程是否处于活跃状态；
- daemon、isDaemon()&setDaemon()：守护线程相关；
    - 守护线程的含义：守护线程会在主线程结束的时候一起结束，无论它有没有执行完毕
    - t.dameon = True 和 t.setDaemon(True)都是将t设置为守护线程，必须在start()之
    前执行
    - 不适用于IDLE环境中的交互模式或者脚本运行模式,如jupyternotebook
- setName()：为线程设置名称
- getName()：获取线程名称

- join([join])：阻塞当前线程，等待被调线程结束或超时后（即让调用了join方法的线程必须同时执行完或超时后才能继续执行之后的代码）再继续执行当前线程的后续代码，参数 timeout 用来指定最长等待时间，单位为秒。
- run()：线程被cpu调度后自动执行线程对象的run方法，如果想自定义线程类，直接重写run方法就行了


### threading提供的方法和属性
- active_count(),activeCount()返回活跃的线程对象数量 
- current_thread(), currentThread():返回当前的Thread对象
- get_ident()：返回当前线程的线程标识符，线程标识符是一个非负整数，并无特殊含义，只是用来标识线程，该整数可能会被循环利用
- enumerate():返回当前活跃状态的所有线程对象列表
- stack_size([size]):返回创建线程时使用的栈的大小，如果指定size参数，则用啊指定后续创建的线程使用的栈的大小size必须是0或者大于大于32000的正整数
- main_thread() 返回主线程对象，即启动Python解释器的线程对象


# 多线程实现同步的方法:锁，条件变量，信号量，事件类

## 线程锁
- 在一个进程中，线程间可以共享向系统申请的内存空间，当同一个资源被多个线程竞争使用时，例如线程共享进程的变量，其就有可能被任何一个线程修改。所以对这种共享资源的访问就需要加上锁来保护，以免该资源对象被改的乱七八糟。


### 互斥锁
- 为了防止上面情况的发生，就出现了互斥锁（LOCK)
- 互斥锁只允许一个线程更改数据
    - lock = threading.Lock():产生一把锁
        - lock.acquire():获取锁
        - lock.release():释放锁
            Lock是 比较低级的同步原语，当被锁定以后不属于特定的线程。一个锁有两种状态: locked和unlocked。 如果锁处于unclocked状态，acquire(方 法将其修改为locked并立即返回;如果锁已处于locked状态，则阻塞当前线程并等待其他线程释放锁然后将其修改为locked并立即返回，或等待一定的时间后返回但不修改锁的状态。release()方法将锁状态由locked修改为unlocked并立即返回，如果锁状态本来已经是unlocked,调用该方法将会抛出异常。
- 线程安全问题
    - 如果一个资源/变量，它对于多线程来说，不用枷锁也不会引起任何问题，则称为线程安全
    - 线程不安全的变量类型：list,set,dict
    - 线程安全变量类型：queue
    
    
### 可重入锁，也称递归锁
- 一个锁，可以被一个线程多次申请
- 主要解决递归调用的时候，需要申请锁的情况
- 与LOCK的使用方法一样，但他支持嵌套，在多个锁没有释放的时候一般会使用RLOCK类
    - lock = threading.RLock()
            RLock与Lock的主要区别：在同一线程内，对RLock进行多次acquire()操作，程序不会阻塞。也就是说，在一个线程内，可以执行多个lock.acquire()，同样当我们想要解除阻塞的时候需要执行同样个数的lock.release()才可以。
            
            
### 信号量（Semaphore和BoundedSemaphore）
#### threading.Semaphore(num)普通信号量
- 控制能够并发执行的线程数，超出的线程先阻塞，直到前面有线程运行完毕再进去执行
    - Semaphore管理一个内置计数器，每当调用acquire（）时内置计数器-1，调用release（）时+1。计数器不能小于0，当计数器为0时，acquire（）将阻塞线程直到其他线程调用release
    - 限制一个时间点内进程的数量为num，保证如果在主机执行IO密集型任务的时候再执行这种类型的程序时，不会出现计算机宕机情况的发生
    - 普通信号量可以无限制释放,记住请求信号量一定要小于等于释放的信号量，否则程序可能无法正常运行
    - semaphore = threading.Semaphore(num)
    - semaphore.acquire()
    - semaphore.release()
    

### threading.BoundedSemaphore(num)限制信号量
- 和普通的信号量一样，有限的信号量内部维护一个计数器，该计数器=initialValue+release-acquire。当 计数器的值为0时，acquire方法调用会被阻塞。计数器初始值为1。
- 限制信号量只能被释放num次,也就是说信号量不能通过释放来大于初始设定值num

In [ ]:
# 没有设置多线程时的运行代码
import time

def loop1():
    start_time = time.time()
    print("Start loop 1 at :", time.ctime(start_time))
    time.sleep(4)
    end_time = time.time()
    print("End loop 1 at:", time.ctime(end_time))
    print("We have spent %.5f seconds"%(end_time - start_time))

def loop2():
    start_time = time.time()
    print("Start loop 2 at :", time.ctime(start_time))
    time.sleep(2)
    end_time = time.time()
    print("End loop 2 at:", time.ctime(end_time))
    print("We have spent %.5f seconds"%(end_time - start_time))

    
def main():
    print("Starting at:", time.ctime())
    loop1()
    loop2()
    print("All done at:", time.ctime())
    
if __name__ == '__main__':
    main()

In [ ]:
# _thread案例，_thread.start_new_thread新建一个线程
import time
import _thread as thread

def loop1():
    start_time = time.time()
    print("Start loop 1 at :", time.ctime(start_time))
    time.sleep(4)
    end_time = time.time()
    print("End loop 1 at:", time.ctime(end_time))
    print("We have spent %.5f seconds"%(end_time - start_time))

def loop2():
    start_time = time.time()
    print("Start loop 2 at :", time.ctime(start_time))
    time.sleep(2)
    end_time = time.time()
    print("End loop 2 at:", time.ctime(end_time))
    print("We have spent %.5f seconds"%(end_time - start_time))

    
def main():
    t1 = time.time()
    print("Starting at:", time.ctime())
    thread.start_new_thread(loop1, ())
    print("----------")
    thread.start_new_thread(loop2, ())
    t2 = time.time()
    print("All done spent {}:", time.ctime())
    
if __name__ == '__main__':
    main()
    
# 注意打印结果，按道理来说主线程运行完毕，子线程就结束不再运行，所以这里的结果又问题
# 使用pycharm

In [ ]:
# threading.Thread创建多线程
import time
from threading import Thread

def loop1(num):
    start_time = time.time()
    print("Start loop 1 at :", time.ctime(start_time))
    time.sleep(4)
    print("hihiahihia {0}".format(num))
    end_time = time.time()
    print("We have spent %.5f seconds"%(end_time - start_time))

def loop2(num):
    start_time = time.time()
    print("Start loop 2 at :", time.ctime(start_time))
    time.sleep(2)
    print("huhahuhahuha {0}".format(num))
    end_time = time.time()
    print("We have spent %.5f seconds"%(end_time - start_time))

    
def main():
    sta = time.time()
    print("Starting at:", time.ctime())
    t1 = Thread(target=loop1, args=("pa",))
    t1.start()
    
    t2 = Thread(target=loop2, args=("pia",))    
    t2.start()
    t1.join()
    t2.join()
    end = time.time()
    print("All done at:", (end - sta))
    
if __name__ == '__main__':
    main()

In [1]:
# 守护线程案例
from threading import Thread
import time
def fun():
    print("Start fun")
    time.sleep(2)
    print("End fun")
    
if __name__  == "__main__":
    print("Main thread")
    t1 = Thread(target=fun, args=())
    t1.setDaemon(True)
    # t1.daemon = True和上面功能一样
    t1.start()
    time.sleep(1)
    print("Main thread end")

Main thread
Start fun
Main thread end
End fun


In [ ]:
# 非守护线程案例
from threading import Thread
import time
def fun():
    print("Start fun")
    time.sleep(2)
    print("End fun")
    
if __name__  == "__main__":
    print("Main thread")
    t1 = Thread(target=fun, args=())
    t1.start()
    time.sleep(1)
    print("Main thread end")
    # 得到的结果和上面一样，是由于daemon在jupyter中不起作用

In [ ]:
# threading.Semaphore(num),num默认值为1
import threading
import time

def fun(semaphore, num):
    # 获得信号量，信号量减一
    """#semaphore.acquire():若加上该语句，则表明信号量再减一，
    而后面又没有释放该信号量，所以后面的线程会一直等待该信号量释放然后拿到它去执行
    ，一直等一直等但他永远不会释放，所有后面的线程就永远不可能执行，所以打印结果只有
    thread 1 is running"""
    semaphore.acquire()
    print("Thread %d is running." % num)
    time.sleep(3)
    # 释放信号量，信号量加一
    semaphore.release()



if __name__ == '__main__':
    # 初始化信号量，数量为2
    semaphore = threading.Semaphore(2)

    # 运行4个线程
    for num in range(6):
        t = threading.Thread(target=fun, args=(semaphore, num))
        t.start()


In [ ]:
# threading.BoundedSemaphore(num)

import threading
import time


def fun(semaphore, num):
    # 获得信号量，信号量减一
    semaphore.acquire()
    print("Thread %d is running." % num)
    time.sleep(3)
    # 释放信号量，信号量加一
    semaphore.release()
    # 再次释放信号量，信号量加一，这是超过限定的信号量数目，这时会报错ValueError: Semaphore released too many times
    #semaphore.release()


if __name__ == '__main__':
    # 初始化信号量，数量为2，最多有2个线程获得信号量，信号量不能通过释放而大于2
    semaphore = threading.BoundedSemaphore(3)

    # 运行4个线程
    for num in range(4):
        t = threading.Thread(target=fun, args=(semaphore, num))
        t.start()


In [ ]:
# 创建线程的第二种方法：继承threading.Thread()

import threading
import time

class MyThread(threading.Thread):
    def __init__(self, arg):
        super(MyThread, self).__init__()
        self.arg = arg
    # 必须重写run函数
    def run(self):
        time.sleep(2)
        print("the args for this class is {0}".format(self.arg))

print(time.ctime())
for i in range(4):
    t = MyThread(i)
    t.start()
    t.join()
print("main thread is done!!!!!!!!!")
print(time.ctime())

In [2]:
# threading.Thread()的工业写法
import threading
import time

loop = [4,2]
class ThreadFunc():
    def __init__(self, name):
        self.name = name
    def loop(self, nloop, nsec):
        """
        param nloop:loop函数的名称
        param nsec:系统休眠时间
        """
        print("Start loop:", nloop, 'at', time.ctime())
        time.sleep(nsec)
    
def main():
    print("Starting at:", time.ctime())
    # ThreadingFunc("loop").loop与 t = ThreadFunc("loop") 再 t.loop相同
    # 传入的是类中的函数
    t = ThreadFunc("loop")
    t1 = threading.Thread(target = t.loop, args=("LOOP one", 3))
    # 下面这种写法更加工业
    t2 = threading.Thread(target = ThreadFunc('loop').loop, args=("LOOP two", 1))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("All done")
    
if __name__ == "__main__":
    main()

Starting at: Sat Feb  6 15:52:28 2021
Start loop:Start loop: LOOP two at  LOOP one at Sat Feb  6 15:52:28 2021
Sat Feb  6 15:52:28 2021
All done


In [ ]:
# threading.Lock():给变量上锁，保护它被使用的唯一性
import time
import threading
sum = 0
loopsum = 1000000
lock = threading.Lock()
def myAdd():
    global sum, loopsum
    for i in range(1, loopsum):
        lock.acquire()
        sum += 1
        lock.release()
def myMinu():
    global sum, loopsum
    for i in range(1, loopsum):
        lock.acquire()
        sum -= 1
        lock.release()


if __name__ == "__main__":
    print("start at:", time.ctime())
    print("starting from {}".format(sum))
    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("Done to {}".format(sum))
    print("end at ", time.ctime())

In [5]:
# 可重入锁（递归锁）示例---RLock
import threading, time

class MyThread(threading.Thread):
    def run(self):
        global num
        time.sleep(1)
        if mutex.acquire(timeout=1):
            num += 1
            msg = self.name + 'set num to ' + str(num)
            print(msg)
            mutex.acquire()
            mutex.release()
            mutex.release()
            
num = 0
mutex = threading.RLock() # 若该处为 mutex = threading.Lock(),则就会报错

def testing():
    for i in range(5):
        t = MyThread()
        t.start()
        
if __name__ == "__main__":
    testing()

Thread-12set num to 1
Thread-13set num to 2
Thread-15set num to 3
Thread-14set num to 4
Thread-16set num to 5



# 生产者消费者模型（主要用于解耦）

            在多线程开发当中，如果生产线程处理速度很快，而消费线程处理速度很慢，那么生产线程就必须等待消费线程处理完，才能继续生产数据。同样的道理，如果消费线程的处理能力大于生产线程，那么消费线程就必须等待生产线程。为了解决这个问题于是引入了生产者和消费者模式
            生产者消费者模式是通过一个容器来解决生产者和消费者的强耦合问题。生产者和消费者彼此之间不直接通讯，而通过阻塞队列来进行通讯，所以生产者生产完数据之后不用等待消费者处理，直接扔给阻塞队列，消费者不找生产者要数据，而是直接从阻塞队列里取，阻塞队列就相当于一个缓冲区，平衡了生产者和消费者的处理能力。

In [6]:
# 生产者消费者模型举例
import threading
import time 
import queue

def producer():
    count = 1
    while True:
        q.put("No.%d"%count)
        print("Producer put No.%d"%count)
        time.sleep(0.5)
        for i in range(10):
            count += i
        
def consumer(name):
    while True:
        print("%s get %s"% (name, q.get()))
        time.sleep(1.5)
        
q = queue.Queue(maxsize=5)
p = threading.Thread(target=producer, args=())
c = threading.Thread(target=consumer, args=("Tom", ))
#p.start()
#c.start()
# 这段代码停不下来

In [ ]:
import queue
import time
class Producer(threading.Thread):
    def run(self):
        global q
        count = 0
        while True:
            if q.qsize() < 1000:
                for i in range(100):
                    count += 1
                    msg = "Producing" + str(count) + "product"
                    q.put(msg)
                    print(msg)
            time.sleep(0.5)
            
class Consumer(threading.Thread):
    def run(self):
        global q
        while True:
            if q.qsize() > 100:
                for i in range(3):
                    msg = self.name + "consums" + q.get()
                    print(msg)
            time.sleep(1)

if __name__ == "__main__":
    q = queue.Queue()
    for i in range(500):
        q.put("初始产品：" + str(i))
    for i in range(2):
        p = Producer()
      """  p.start()
    for i in range(5):
        c = Consumer()
        c.start()
    print(threading.active_count())"""

# 多线程死锁问题
- 死锁的一个原因是互斥锁。两个线程互相等待对方的锁，互相占用着资源不放。归根结底，还是由于线程同时获取多个锁造成的。
- 比如：一个线程获取了第一个锁，然后在获取第二个锁的时候发生阻塞，那么这个线程就可能阻塞其他线程的执行，从而导致整个程序假死。
- 死锁是每一个多线程程序都会面临的一个问题。根据经验来讲，尽可能保证每一个线程只能同时保持一个锁，这样程序就不会被死锁。
- 解决死锁的一种方法是：在进程获取锁的时候严格按照对象id升序排列获取
- 解决死锁问题没有什么太优雅的办法，凭借经验


In [ ]:
# 死锁实例
import threading,time
lock_1 = threading.Lock()
lock_2 = threading.Lock()

def func_1():
    print("func 1 starting..............")
    lock_1.acquire()
    print("func_1 申请了lock_1....")
    time.sleep(2)#延长程序时间，保证可以产生死锁
    print("func 1 等待 lock_2......")
    lock_2.acquire()
    print("func 1 申请了 lock_2.......")
    lock_2.release()
    print("func_1 释放了 lock_2......")
    lock_1.release()
    print("func_1 释放了 lock_1......")
    
def func_2():
    print("func 2 starting..............")
    lock_2.acquire()
    print("func_2 申请了lock_2....")
    time.sleep(4)#延长程序时间，保证可以产生死锁
    print("func 2 等待 lock_1......")
    lock_1.acquire()
    print("func 2 申请了 lock_1.......")
    lock_1.release()
    print("func_2 释放了 lock_1......")
    lock_2.release()
    print("func_2 释放了 lock_2......")

    
t1 = threading.Thread(target=func_1, args=())
t2 = threading.Thread(target=func_2, args=())
t1.start()
t2.start()
print("All done")

# 多线程事件（Event类）：同步条件
- threading.Event()
- 同步：当线程在系统中运行时，线程的调度具有一定的透明性，通常程序无法准确控制线程的轮换执行。因此，如果有需要，可通过线程通信来保证线程协调运行，即达到线程同步的目的。
- 实现同步的方法：Event
- Event：是线程间通信机制之一：一个线程发送一个event信号，其他的线程则等待这个信号。常用在一个线程需要根据另外一个线程的状态来确定自己的下一步操作的情况。
- Event原理：
    - 通过生成一个“事件”对象（可以看成是一个传令兵）来管理内部的一个标志，传令兵通过set()方法将标志设置为True，并使用clear()方法将标志设置为False。wait（）方法就是号令所有线程等待，即阻塞，直到标志变成True（标志要靠set方法将其变为True），才继续进行下面的代码。该标志初始为False
    

### Event类常用方法：
    - is_set()：当且仅当内部标志为True时返回True。
    - set()：将内部标志设置为True。所有等待它成为True的线程都被唤醒。当标志保持在True的状态时，线程调用wait()是不会阻塞的（好比传令兵发信号要成员去攻打敌人，即set将标志设置为True，如果没有把该信号清除，然后只是大叫要成员等待，那么成员是不会停止的，他们只认信号。除非用clear()重置，wait才能生效）。
    - clear()：将内部标志重置为False。随后，调用wait()的线程将阻塞，直到另一个线程调用set()将内部标志重新设置为True。
    - wait(timeout=None)：阻塞直到内部标志变成真。如果内部标志在wait()方法调用时为True，则立即返回。否则，则阻塞，直到另一个线程调用set()将标志设置为True，或发生超时。该方法总是返回True，除非设置了timeout并发生超时。

In [ ]:
# Event类示例:直接调用threading.Thread
import threading, time

event = threading.Event()
def chihuoguo(name):
    print("{} 已经启动".format(threading.current_thread().getName()))
    print("小伙伴 %s 已经进入就餐状态" % name)
    time.sleep(1)
    event.wait()
    print("%s 收到通知了"% threading.current_thread().getName())
    print("小伙伴 {} 开始动筷子了".format(name))
    
t1 = threading.Thread(target=chihuoguo, args=('xiaoming', ))
t2 = threading.Thread(target=chihuoguo, args=('xiaobai', ))
t1.start()
t2.start()                                  
time.sleep(1)
print("主线程告诉大家可以开始吃喽")
event.set()

In [ ]:
# Event类示例:继承threading.Thread
event = threading.Event()

def chihuoguo(name):
    print("{} 已经启动".format(threading.current_thread().getName()))
    print("小伙伴 %s 已经进入就餐状态" % name)
    time.sleep(1)
    event.wait()
    print("%s 收到通知了"% threading.current_thread().getName())
    print("小伙伴 {} 开始动筷子了".format(name))
    
    
class Myhuoguo(threading.Thread):
    def __init__(self,name):
        threading.Thread.__init__(self)
        self.people = name
    
    def run(self):
        chihuoguo(self.people)
        print("结束线程：{}".format(threading.current_thread().getName()))
        
for i in ['xb', 'xh', 'xc']:
    t = Myhuoguo(i)
    t.start()
time.sleep(0.2)
print("都到齐了，大家开始吃吧")
event.set()

In [ ]:
# Event类wait中timeout超时案例
import threading, time

event = threading.Event()
def chihuoguo(name):
    print("{} 已经启动".format(threading.current_thread().getName()))
    print("小伙伴 %s 已经进入就餐状态" % name)
    time.sleep(1)
    event.wait(timeout=0.1)
    print("%s 收到通知了"% threading.current_thread().getName())
    print("小伙伴 {} 开始动筷子了".format(name))


t1 = threading.Thread(target=chihuoguo, args=('xiaoming', ))
t2 = threading.Thread(target=chihuoguo, args=('xiaobai', ))
t1.start()
t2.start()                                  
time.sleep(3)
print("主线程告诉大家可以开始吃喽")
event.set()
# 可以看出最后才打印主线程告诉大家可以开始吃了，因为wait超时，就直接运行下去了
# 就好比约定大家几点开始一起进攻，但是主帅睡过头了，其他人不管他就按照约定去攻打了

In [3]:
# 一旦标志被set激活，而又没clear，则wait不会阻塞
import threading, time

event = threading.Event()
def chihuoguo(name):
    print("{} 已经启动".format(threading.current_thread().getName()))
    print("小伙伴 %s 已经进入就餐状态" % name)
    time.sleep(1)
    event.wait()
    print("%s 收到通知了"% threading.current_thread().getName())
    print("小伙伴 {} 开始动筷子了".format(name))

event.set()
t1 = threading.Thread(target=chihuoguo, args=('xiaoming', ))
t2 = threading.Thread(target=chihuoguo, args=('xiaobai', ))
t1.start()
t2.start()                                  
time.sleep(1)
print("主线程告诉大家可以开始吃喽")
event.set()

Thread-9 已经启动
小伙伴 xiaoming 已经进入就餐状态
Thread-10 已经启动
小伙伴 xiaobai 已经进入就餐状态
Thread-9 收到通知了
主线程告诉大家可以开始吃喽小伙伴 xiaoming 开始动筷子了Thread-10 收到通知了
小伙伴 xiaobai 开始动筷子了




# 多线程条件类（Condition）:条件变量同步
- 满足条件之后才能够执行
- class threading.Condition(lock=None)
    - 这个类实现条件变量对象。条件变量允许一个或多个线程等待，直到它们被另一个线程唤醒。
    - 如果给出了lock参数而不是None，则它必须是Lcok或RLock对象，并以它作为底层的锁。否则将默认创建一个RLock对象。
    - Condition遵循上下文管理协议。
    - 和Event事件类差不多，只是多了锁功能

- 实例方法
    - acquire()：申请锁
    - release()
        - 释放锁。这个方法调用底层锁的相应方法。

    - wait(timeout=None)
        - 当前线程处于等待状态，并且会释放锁，可以被其他线程使用notify或者notify_all唤醒，被唤醒后等待上锁，上锁后继续执行下面的代码
        - 线程挂起，等待被唤醒(其他线程的notify方法)或者发生超时。调用该方法的线程必须先获得锁，否则引发RuntimeError。
        该方法会释放底层锁，然后阻塞，直到它被另一个线程中的相同条件变量的notify()或notify_all()方法唤醒，或者发生超时。一旦被唤醒或超时，它会重新获取锁并返回。
        返回值为True，如果给定timeout并发生超时，则返回False。

    - wait_for(predicate, timeout=None)
        - 等待直到条件变量的返回值为True。predicate应该是一个返回值可以解释为布尔值的可调用对象。可以设置timeout以给定最大等待时间。
        - 该方法可以重复调用wait()，直到predicate的返回值解释为True，或发生超时。该方法的返回值就是predicate的最后一个返回值，如果发生超时，返回值为False。它与wait()的规则相同：调用前必须先获取锁，阻塞时释放锁，并在被唤醒时重新获取锁并返回。
    - notify(n=1)
        - 通知某个正在等待的线程，默认是1个等待的线程
        - 默认情况下，唤醒等待此条件变量的一个线程(如果有)。调用该方法的线程必须先获得锁，否则引发RuntimeError。
        - 该方法最多唤醒n个等待中的线程，如果没有线程在等待，它就是要给无动作的操作。
        - 注意：要被唤醒的线程实际上不会马上从wait()方法返回(唤醒)，而是等到它重新获取锁。这是因为notify()并不会释放锁，需要线程本身来释放(通过wait()或者release())

     - notify_all()
         - 此方法类似于notify()，但唤醒的是所有等待的线程。
         - notify和notify_all是不会释放锁的，并且在release之前使用



In [4]:
# 多线程条件类示例：threading.Condition()
import threading
import time


num = 0
con = threading.Condition()


class Producer(threading.Thread):
    """生产者"""
    def run(self):
        global num
        # 获取锁
        con.acquire()
        while True:
            num += 1
            print('生产了1个，现在有{0}个'.format(num))
            time.sleep(1)
            if num >= 5:
                print('已达到5个，不再生产')
                # 唤醒消费者
                con.notify()
                # 等待-释放锁；被唤醒-获取锁
                con.wait()
        # 释放锁
        con.release()


class Customer(threading.Thread):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.money = 3

    def run(self):
        global num
        while self.money > 0:
            # 由于场景是多个消费者进行抢购，如果将获取锁操作放在循环外(如生产者),
            # 那么一个消费者线程被唤醒时会锁住整个循环，无法实现另一个消费者的抢购。
            # 在循环中添加一套"获取锁-释放锁",一个消费者购买完成后释放锁，其他消费者
            # 就可以获取锁来参与购买。
            con.acquire()
            if num <= 0:
                print('没货了，{0}通知生产者'.format(
                    threading.current_thread().name))
                con.notify()
                con.wait()
            self.money -= 1
            num -= 1
            print('{0}消费了1个, 剩余{1}个'.format(
                threading.current_thread().name, num))
            con.release()
            time.sleep(1)
        print('{0}没钱了-回老家'.format(threading.current_thread().name))


if __name__ == '__main__':
    p = Producer(daemon=True)
    c1 = Customer(name='Customer-1')
    c2 = Customer(name='Customer-2')
    p.start()
    c1.start()
    c2.start()
    c1.join()
    c2.join()

生产了1个，现在有1个
生产了1个，现在有2个
生产了1个，现在有3个
生产了1个，现在有4个
生产了1个，现在有5个
已达到5个，不再生产
Customer-1消费了1个, 剩余4个
Customer-2消费了1个, 剩余3个
Customer-1消费了1个, 剩余2个
Customer-2消费了1个, 剩余1个
Customer-1消费了1个, 剩余0个
没货了，Customer-2通知生产者
生产了1个，现在有1个
Customer-1没钱了-回老家
生产了1个，现在有2个
生产了1个，现在有3个
生产了1个，现在有4个
生产了1个，现在有5个
已达到5个，不再生产
Customer-2消费了1个, 剩余4个
Customer-2没钱了-回老家


In [ ]:
# condition案例2
import threading
import random
import time

gProduct = 1000
gCondition = threading.Condition()
gTotalTimes = 10
gTimes = 0


class Producer(threading.Thread):
    def run(self):
        global gTimes
        global gProduct
        while True:
            gCondition.acquire()
            product = random.randint(100, 1000)
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gProduct += product
            print(f"第{gTimes + 1}次生产：{threading.current_thread().getName()}生产了{product}件产品，现有{gProduct}件产品")
            gTimes += 1
            # 通知等待的线程，但不会释放锁
            gCondition.notify_all()
            # 需要手动释放锁
            gCondition.release()
            time.sleep(0.5)


class Consumer(threading.Thread):
    def run(self):
        global gTimes
        global gProduct
        while True:
            gCondition.acquire()
            product = random.randint(100, 1000)
            while gProduct < product:
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return
                print("商品正在备货中，请耐心等待")
                gCondition.wait()
            gProduct -= product
            print(f"{threading.current_thread()} 消费了{product}件产品, 还剩余{gProduct}件产品")
            gCondition.release()
            time.sleep(0.5)

            


def main():
    # 5个消费者
    for i in range(5):
        c = Consumer(name=f"消费者{i}")
        c.start()
    # 4个生产者
    for i in range(4):
        p = Producer(name=f"生产者{i}")
        p.start()

if __name__ == "__main__":
    main()
# 不知道为啥运行不成功，pycharm可以

In [ ]:
import threading
import random
import time

gProduct = 1000
gTotalTimes = 10
gTimes = 0
gCondition = threading.Condition()


class Producer(threading.Thread):
    def run(self):
        global gProduct
        global gTimes
        while True:
            new_product = random.randint(100, 1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gProduct += new_product
            print(f"第{gTimes + 1}次生产：{threading.current_thread().getName()}生产了{new_product}件产品，现有{gProduct}件产品")
            gTimes += 1
            gCondition.notify_all()
            gCondition.release()
            time.sleep(0.5)


class Consumer(threading.Thread):
    def run(self):
        global gProduct
        global gTimes
        while True:
            con_prod = random.randint(100, 1000)
            gCondition.acquire()
            while gProduct < con_prod:
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    # 使用return直接退出循环，而break的话只退出一层循环
                    return
                print("商品不足，正在备货中，请耐心等待")
                gCondition.wait()
            gProduct -= con_prod
            print(f"{threading.current_thread()}消费了{con_prod}件产品，还剩余{gProduct}件产品")
            gCondition.release()
            time.sleep(0.4)


def main():
    for i in range(5):
        p = Producer(name=f"生产者{i}")
        p.start()

    for i in range(5):
        c = Consumer(name=f"消费者{i}")
        c.start()


if __name__ == "__main__":
    main()

# 多线程定时类（Timer）:threading.Timer(interval, function)
- 定时器 就是隔多长时间去触发任务执行,指定n秒后执行某操作,可以使用cancel提前取消。
- interval 第一个参数传 间隔时间;function  传执行任务的函数  隔了多少秒后执行这个函数
- Timer从Thread派生，没有增加实例方法。

In [ ]:
from threading import Timer
 
def hello():
    print("hello, world")

t = Timer(1, hello)
t.start()  # after 1 seconds, "hello, world" will be printed